In [1]:
import pandas as pd
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np

In [2]:
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [3]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(None, string.punctuation)
    filter_str = final_str.translate(None, digits)
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stopwords.words('english'):
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()

In [29]:
df_train = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/train.csv')
df_train['tag'] = 'train'
print df_train.shape

(43266, 10)


In [30]:
df_test = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/test.csv') 
df_test['tag'] = 'test'
print df_test.shape

(18543, 9)


In [31]:
df = pd.concat([df_train],axis= 0)
print df.shape
df.head()

(43266, 10)


,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Complaint-Status,Consumer-disputes,Consumer-complaint-summary,tag
0,Tr-1,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Closed with explanation,Yes,"Seterus, Inc a déposé un faux rapport auprès d...",train
1,Tr-2,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,Closed with non-monetary relief,No,XX / XX / XXXX La requête en faillite n ° XXXX...,train
2,Tr-3,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,Closed with explanation,No,"El XXXX / XXXX / 15, estaba preparando el vuel...",train
3,Tr-4,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,Closed with explanation,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea...",train
4,Tr-5,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,Closed with explanation,No,J'ai obtenu un compte de crédit de soins pour ...,train


In [32]:
(df['Transaction-Type'].value_counts())

Debt collection                                                                 10422
Mortgage                                                                         7950
Credit reporting                                                                 6706
Credit card                                                                      4119
Bank account or service                                                          3264
Credit reporting, credit repair services, or other personal consumer reports     3169
Student loan                                                                     2874
Consumer Loan                                                                    2033
Credit card or prepaid card                                                       719
Checking or savings account                                                       496
Payday loan                                                                       363
Money transfers                                       

In [33]:
tran_type = {'Debt collection' : 'c' ,
             'Credit reporting':'c' ,
             'Credit card':'c',
             'Credit reporting, credit repair services, or other personal consumer reports':'c',
             'Credit card or prepaid card':'c',\
             'Prepaid card':'c',
             'Mortgage':'l',
             'Student loan':'l',
             'Consumer Loan':'l',
             'Payday loan':'l',
             'Vehicle loan or lease':'l',\
             'Payday loan, title loan, or personal loan':'l',\
             'Virtual currency':'c',\
             'Money transfer, virtual currency, or money service':'c',\
             'Money transfers':'c',\
             'Bank account or service':'b',\
             'Checking or savings account':'b',
             'Other financial service':'o'
    }
len(tran_type)

18

In [34]:
replace = []
for t in df['Transaction-Type']:
    replace.append(tran_type[t])

In [35]:
df['Transaction-Type'] = replace

In [36]:
df['Transaction-Type'].value_counts()

c    25926
l    13520
b     3760
o       60
Name: Transaction-Type, dtype: int64

In [ ]:
from datetime import datetime
def convert_in_date(s):
    datetime_object = datetime.strptime(s,'%m/%d/%Y')
    return datetime_object

In [ ]:
df['Date-received'] = df['Date-received'].apply(convert_in_date)

In [ ]:
df['Date-sent-to-company'] = df['Date-sent-to-company'].apply(convert_in_date)

In [ ]:
date_equal = df.loc[df['Date-sent-to-company'] == df['Date-received']]

In [ ]:
# date_equal[['Date-received','Date-sent-to-company','Complaint-Status','Consumer-disputes','Transaction-Type']]

In [ ]:
df['Transaction-Type'] = df['Transaction-Type'].map(lambda x : x.lower().replace(' ','_'))

In [ ]:
df['Complaint-reason'] = df['Complaint-reason'].apply(preprocess_data)

In [ ]:
text = list(df['Complaint-reason'])
vectorizer = TfidfVectorizer(min_df=1)
# tokenize and build vocab
vectorizer.fit(text)

In [ ]:
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)

In [ ]:
# import time

# from sklearn.manifold import TSNE

# n_sne = 7000
# time_start = time.time()
# tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
# tsne_results = tsne.fit_transform(vector.toarray())


In [37]:
vec_df = pd.DataFrame()

In [38]:
print vec_df.shape
vec_df.head()

(0, 0)


""


In [39]:
vec_df['tag'] = list(df['tag'])
vec_df['Complaint-Status'] = list(df['Complaint-Status'])
vec_df['Transaction-Type'] = list(df['Transaction-Type'])
vec_df['Complaint-ID'] = list(df['Complaint-ID'])
#vec_df['cd'] = list(df['Consumer-disputes'])

In [40]:
vec_df.shape

(43266, 4)

In [41]:
complete_df_one_hot = pd.get_dummies(data=vec_df, columns=['Transaction-Type'])
#complete_df_one_hot = vec_df

In [42]:
train_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'train']
train_data.drop(['tag'],inplace=True,axis=1)
test_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'test']
test_data.drop(['tag'],inplace=True,axis =1)


In [43]:
train_data.shape

(43266, 6)

In [44]:
train_data.head()

,Complaint-Status,Complaint-ID,Transaction-Type_b,Transaction-Type_c,Transaction-Type_l,Transaction-Type_o
0,Closed with explanation,Tr-1,0,0,1,0
1,Closed with non-monetary relief,Tr-2,0,1,0,0
2,Closed with explanation,Tr-3,1,0,0,0
3,Closed with explanation,Tr-4,0,1,0,0
4,Closed with explanation,Tr-5,0,1,0,0


In [45]:
y_train = (train_data['Complaint-Status'])
x_train = train_data.drop(['Complaint-Status','Complaint-ID'],axis =1)

In [46]:
# pd.concat([x_train,feature_tt])
print x_train.shape
x_train.head()

(43266, 4)


,Transaction-Type_b,Transaction-Type_c,Transaction-Type_l,Transaction-Type_o
0,0,0,1,0
1,0,1,0,0
2,1,0,0,0
3,0,1,0,0
4,0,1,0,0


In [47]:
print y_train.shape
y_train_one_hot = pd.get_dummies(y_train,prefix=['Complaint-Status'])
print y_train_one_hot.shape

(43266,)
(43266, 5)


In [48]:
print x_train.shape
print y_train.shape

(43266, 4)
(43266,)


In [58]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=2)
X_train_res, y_train_res = (x_train, y_train.ravel())


In [59]:
print X_train_res.shape
print y_train_res.shape

(43266, 4)
(43266,)


In [60]:
import time

from sklearn.manifold import TSNE

n_sne = 7000
time_start = time.time()
tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(X_train_res)


[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 43266 samples in 1.150s...
[t-SNE] Computed neighbors for 43266 samples in 8.219s...
[t-SNE] Computed conditional probabilities for sample 1000 / 43266
[t-SNE] Computed conditional probabilities for sample 2000 / 43266
[t-SNE] Computed conditional probabilities for sample 3000 / 43266
[t-SNE] Computed conditional probabilities for sample 4000 / 43266
[t-SNE] Computed conditional probabilities for sample 5000 / 43266
[t-SNE] Computed conditional probabilities for sample 6000 / 43266
[t-SNE] Computed conditional probabilities for sample 7000 / 43266
[t-SNE] Computed conditional probabilities for sample 8000 / 43266
[t-SNE] Computed conditional probabilities for sample 9000 / 43266
[t-SNE] Computed conditional probabilities for sample 10000 / 43266
[t-SNE] Computed conditional probabilities for sample 11000 / 43266
[t-SNE] Computed conditional probabilities for sample 12000 / 43266
[t-SNE] Computed conditional probabilities for sa

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train_internal, y_test_internal = train_test_split(X_train_res,y_train_res ,random_state = 0)


In [57]:
gb = MultinomialNB().fit(X_train, y_train_internal)
pred_internal = gb.predict(X_test)
import sklearn
sklearn.metrics.f1_score(y_test_internal, pred_internal,average='weighted')

0.20932327718959093

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 500, random_state = 42)

# Train the model on training data
rm = rf.fit(X_train, y_train_internal)
pred_internal = rm.predict(X_test)



In [ ]:
import sklearn
sklearn.metrics.f1_score(y_test_internal, pred_internal,average='weighted')

In [ ]:
from sklearn.svm import SVC
sv = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
sv.fit(X_train, y_train_internal)
pred_internal = sv.predict(X_test)

In [ ]:
import sklearn
sklearn.metrics.f1_score(y_test_internal, pred_internal,average='weighted')

In [ ]:
from tpot import TPOTClassifier
tpot = TPOTClassifier(generations = 8, population_size = 50, verbosity=2,scoring= 'f1_weighted')
tpot.fit(X_train, y_train_internal)